In [30]:
from tqdm import tqdm 
import pandas as pd 

airports     = pd.read_csv('../us_large_airports.csv')
big_airports =   set(airports['iata_code'].unique()) # {'ATL', 'JFK', 'LAX', 'ORD'} 


flights  = pd.read_csv('../Airports2.csv', usecols=['Origin_airport', 'Destination_airport', 'Origin_city', 'Destination_city', 'Fly_date'])
flights = flights[flights.apply(lambda x: (x['Origin_airport'] in big_airports) and (x['Destination_airport'] in big_airports), axis=1)]
flights  = flights.drop_duplicates()

flights['Fly_date'] = pd.to_datetime(flights['Fly_date'])               

flights.sort_values(by=['Fly_date'], inplace=True)

In [31]:
T = flights['Fly_date'].drop_duplicates().sort_values()
T.index = range(len(T))

## Exhaustive Walk

Results in combinatorial explosion, even with 4 airports. 

In [3]:
# result = pd.DataFrame()
# for i, t in tqdm(T.iteritems(), total=len(T)):
#     subset = flights.loc[flights['Fly_date'] == t , ['Origin_airport', 'Destination_airport']]
#     subset.columns = ['s_%s' % i, 'd_%s' % i]

#     if len(result) == 0:
#         result = pd.concat([result, subset], axis=1)
#     else:
#         result = pd.merge(result, subset, how='left', left_on= "d_%s" % str(i-1) , right_on="s_%s" % str(i))
#         del result["s_%s" % str(i)]
    

## Random Walk

In [33]:
result = pd.DataFrame()
n_walks = 100
walks = []

for _ in tqdm(range(n_walks)):
    walk = []
    for i, t in T.iteritems():

        flights_t = flights.loc[flights['Fly_date'] == t , ['Origin_airport', 'Destination_airport']]
        
        if i == 0:
            s = flights_t.sample(1)['Origin_airport'].values[0]
            walk.append(s)
        
        flights_s = flights_t.loc[flights_t['Origin_airport']==s]
        if len(flights_s) == 0:
            break
        
        d = flights_s['Destination_airport'].sample(1).values[0]
        walk.append(d)
        s = d
    
    walks.append(walk)

100%|██████████| 100/100 [00:45<00:00,  2.22it/s]


In [38]:
walks = pd.DataFrame(walks)
# walks = walks.replace({'ATL': 0, 'JFK': 1, 'LAX': 2, 'ORD': 3})
walks

,0,1,2,3,4,5,6,7,8,9,...,231,232,233,234,235,236,237,238,239,240
0,BWI,DCA,CHS,LGA,BUF,DTW,PHX,IND,IND,MCI,...,None,None,None,None,None,None,None,None,None,None
1,MCO,STL,MSP,IND,MKE,BOS,RIC,LGA,JAX,GSP,...,None,None,None,None,None,None,None,None,None,None
2,CVG,DTW,LAS,TUS,SFO,PIT,1,MSY,CVG,SEA,...,None,None,None,None,None,None,None,None,None,None
3,CMH,CLT,BHM,PHL,BWI,CMH,DFW,LBB,DAL,SAT,...,None,None,None,None,None,None,None,None,None,None
4,MCI,0,TOL,PIT,ALB,DCA,BUF,LGA,DFW,MOB,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PVD,DCA,GRR,MKE,0,AGS,CAE,LGA,TPA,0,...,MSP,LSE,3,LEX,CLT,CHO,LGA,GRR,MKE,PHL
96,DFW,PDX,CLM,SEA,IAH,JAX,IAD,BGM,PIT,CLT,...,LAS,ELP,CNM,HOB,CNM,HOB,CNM,HOB,CNM,ELP
97,STL,MIA,IND,DTW,ALB,CLE,LAS,2,MKE,GRB,...,SYR,MHT,MDW,IND,SEA,BKL,3,HSV,DFW,MLU
98,DFW,ANC,SLC,IAD,JAX,CHS,LGA,CHS,RDU,LEX,...,0,ANC,DFW,MSY,FLL,MCI,CLL,LBB,ABQ,MSP


In [57]:
walks = walks.dropna()

In [58]:
mapping = pd.Series(index=list(big_airports)[1:], data=range(len(big_airports)-1)).to_dict()

walks = walks.replace(mapping)

In [35]:
# walks[[1, 2]].apply(lambda x: tuple(x), axis=1).value_counts()

In [60]:
walks.loc[:, :20].to_csv('walks.csv', index=False)
# walks.to_csv('walks.csv', index=False)

## TODO: Come back to this

It appears that for the same month, there exist multiple flights between a given origin and destination, each with a different number of passengers and seats. 

In [48]:
flights = pd.read_csv('../Airports2.csv')#
flights[(subset['Origin_airport'] == 'LAX') & (flights['Destination_airport'] == 'HNL')]

LAX_HNL    35
LAX_SFO    34
SFO_LAX    33
HNL_LAX    30
JFK_LAX    28
           ..
MSP_CPR     1
MKE_GRB     1
DTW_BGR     1
CMH_CLT     1
CHA_GSO     1
Length: 3453, dtype: int64